# Notions avancées du stockage

## Caractéristiques de vos données
### Taille
Au second chapitre, nous avons vu comment obtenir le nombre de
fichiers de nos données locales, de même que la taille totale.

Sur une grappe de calcul, un **système de fichiers parallel**
[*Lustre*](https://docs.alliancecan.ca/wiki/Tuning_Lustre/fr) ou *GPFS*
est surtout optimisé pour des fichiers de plus de 10 Mo,
c'est-à-dire pour des accès à large bande-passante. Ainsi, à l'échelle des
[grappes de calcul](https://docs.alliancecan.ca/wiki/B%C3%A9luga#Stockage) :

* 1 Ko - 100 Ko : très petit fichier
* 1 Mo - 10 Mo : petit fichier
* **100 Mo - 1 Go : taille idéale**
* 10 Go - 100 Go : gros fichier
* 1 To - 10 To : très gros fichier

### Nombre
À quantité totale égale d'octets, il est plus **difficile pour *Lustre*
de gérer un grand nombre** de petits fichiers qu'un plus petit nombre
de grands fichiers. Ainsi, du point de vue du système de stockage :

* Par dossier :
  * **1 à 100 fichiers par dossier : c'est raisonnable**
  * 300 à 3k fichiers par dossier : une lourdeur se fait sentir
  * 10k fichiers et plus : il faut classer les fichiers dans des sous-dossiers
* Par espace [limité par un quota](https://docs.alliancecan.ca/wiki/Storage_and_file_management/fr#Quotas_et_politiques) :
  * **1 à 500k fichiers : sauvegarde facilement à jour**
  * 1M à 5M fichiers : sauvegarde potentiellement plus longue que 24h
  * 10M fichiers et plus : sauvegarde plus longue que 24h

### Transférabilité
#### Regroupement dans un fichier archive
Étant donné que le transfert ou la synchronisation de
[plusieurs petits fichiers](https://docs.alliancecan.ca/wiki/Handling_large_collections_of_files/fr)
implique un accès fréquent aux métadonnées du système de stockage
*Lustre*, il est donc préférable de
[regrouper les fichiers](https://docs.alliancecan.ca/wiki/Archiving_and_compressing_files/fr)
d'un jeu de données avant tout transfert. Pour ce faire, il existe
plusieurs formats décodables sur les différentes grappes :

* **WinZip** ou **Zip** (`.zip`) - compressé
* **7-Zip** (`.7z`) - compressé
* **Tape ARchive** ou [**TAR**](https://docs.alliancecan.ca/wiki/A_tutorial_on_%27tar%27/fr) (`.tar`) - non compressé, sauf si utilisé avec Gzip ou autre
* **Disk ARchive** ou [**DAR**](https://docs.alliancecan.ca/wiki/Dar/fr) (`.dar`) - compression configurable

**Démonstration** - Création d'une archive DAR

Dans un terminal :

```Bash
dar -w -c arch -g donnees -g images
```
* `-w` : pour taire un avertissement d'écrasement d'archive existante
* `-c` : pour créer l'archive `arch`
* `-g` : pour spécifier un fichier ou dossier à la fois

```Bash
ls arch*
dar -l arch
```
* `-l` : pour lister les fichiers dans l'archive DAR `arch`
* Note : **ne pas** fournir le nom d'une découpe avec l'extension `.1.dar`

```Bash
dar -w -z -c arch -g donnees -g images
```
* `-z` : pour activer la compression pour tous les fichiers
* Voir le résultat avec `dar -l arch`

```Bash
dar -w -z -Z '*.png' -c arch -g donnees -g images
```
* `-Z '*.png'` : pour exclure les fichiers PNG
* Voir le résultat avec `dar -l arch`

#### Compression des fichiers
La [compression des fichiers](https://docs.alliancecan.ca/wiki/Archiving_and_compressing_files/fr)
a un **coût en temps CPU** à chaque fois que nous voulons lire ou écrire
les données. Ainsi, il vaut mieux utiliser la compression **lorsque
c'est "avantageux"** en espace et/ou en temps. Par exemple :
* **Décompresser de nombreux fichiers sur le noeud de calcul**
  plutôt que de lire chaque petit fichier de *Lustre*.

Lorsque le **système de fichiers compresse automatiquement**
vos données avant de les écrire sur le système de stockage :
  * La commande `du` montre l'espace réellement utilisée sur le disque
  * La commande `du --apparent-size` montre la taille normale du fichier

**Démonstration** - Espace utilisé

Dans un terminal :
```Bash
du -s ~
du -s --apparent-size ~
```

* Noter les différences, s'il y en a
* Cette commande peut surcharger le système de stockage;
  le survol récursif des fichiers constitue un grand nombre
  d'accès aux métadonnées de *Lustre*

En pratique, les grappes de calcul offrent des
[outils efficaces](https://docs.alliancecan.ca/wiki/Diskusage_Explorer)
pour mesurer l'espace utilisé en fonction de l'espace alloué.

## Fichiers épars (sparse files)
Restez à l'affût des fichiers épars dont la faible taille sur le disque n'est pas représentative de la taille normale du fichier :
* Une certaine taille `T` est allouée pour le fichier.
  Par exemple : 1 To
* Seulement `t` octets sont réellement écrits dans le fichier
  (où `t << T`). Par exemple : 1 Go
* Ce type de fichiers est typiquement utilisé pour des volumes de
  stockage de machines virtuelles, ce qui économise de l'espace sur le
  serveur hôte
* Problème : lors d'un transfert ou d'une sauvegarde (*backup*),
  ce sont les `T` octets qui sont lus, ce qui peut être catastrophique
  si `T` est vraiment très grand
* **Solution** : les fichiers `.dar` permettent d'optimiser
  l'encapsulation de fichiers épars avant un transfert ou une copie

## Plan de gestion des données actives
Les données sur les grappes de calcul sont généralement présentes en attente d'être traitées, manipulées ou publiées, d'où la notion de *données actives*. Or, il y a une planification à faire et des questions à se poser.

### Vie des données
* **Durée de vie variable** - avons-nous affaire à des données temporaires ou à des données persistantes?
  * Utilisées (à répétition ou non) pendant une seule tâche de calcul?
  * Utilisées entre deux tâches de calcul?
  * Utilisées pour plusieurs calculs par plusieurs personnes?
  * Peut-on importer à nouveau les données en cas de perte?
  * Peut-on recalculer les données en cas de perte? Si oui, à quel coût?
* **Fréquence d'accès** - combien de temps sépare les différents accès?
  * Utilisation courante (à chaque jour, semaine ou mois)?
  * Données à garder pour plus tard, par exemple pour reproduire les résultats?
  * Données à publier sur des dépôts de données de recherche?
* **Urgence d'accès** - à propos de données critiques ou importantes :
  * Combien y a-t-il de copies?
  * Doivent-elles être accessibles en tout temps?

Si aucune planification n'est faite, les données temporaires s'accumulent et prennent de la place dans l'espace alloué qui est de taille limitée. Si les données sont mal décrites, leur identification, leur compréhension et leur publication deviennent des tâches laborieuses pour les personnes qui en héritent.
```Bash
cat .gitignore  # Fichiers à ignorer, car temporaires
cat README.md   # Description du projet
```

### Niveaux d'accès
* Quel est le **niveau de confidentialité** de vos données?
  * **Qui** devrait avoir accès à quelles données?
  * **Où** pouvez-vous héberger vos données?
  * **Quand** : qui devrait avoir accès à vos données à votre départ?
    Et à partir de quand peut-on publier les données?

Référez-vous aux **politiques de votre université** concernant la gestion
des données de recherche.
Sans précaution, les données que vous laissez sur les grappes de
calcul risquent de ne pas être accessibles aux membres d'un
laboratoire qui poursuivent leur recherche.

Dès que possible, mettez en place les accès nécessaires pour le
[partage de vos données](https://docs.alliancecan.ca/wiki/Sharing_data/fr).

### Exemple de partage de dossier:
```Bash
# Enlever l'accès en lecture
chmod g-r $SCRATCH
ls -ld $SCRATCH

# Modifier le GID du répertoire
id
chgrp def-sponsor00 $SCRATCH
ls -ld $SCRATCH

# Création d'un dossier de résultats partagés
mkdir -p $SCRATCH/partage/resultats
ls -lR $SCRATCH

# Ajouter les permissions r (et x, s'il y a lieu) récursivement
chmod -R o+rX $SCRATCH/partage
ls -lR $SCRATCH

# Visiter le dossier partagé de quelqu'un d'autre
cd /scratch/userXY/partage
pwd
ls -l
cd
```